# Dealing with Data
This Notebook will show how to load data via PyTecplot and extract useful information about the data.  You'll learn the basics of loading data and data access.

In [1]:
import tecplot as tp
tp.session.connect(port=7600) # Connect to a live Tecplot 360 session
tp.new_layout() # Make sure we're starting from scratch

Connecting to Tecplot 360 TecUtil Server on:
    tcp://localhost:7600
Connection established.


## Loading Data
Tecplot 360 and therefore PyTecplot have many data loaders, available under the tecplot.data namespace.  Here we use 'load_tecplot' to load the data.

In [2]:
import os
examples_dir = tp.session.tecplot_examples_directory()
datafile = os.path.join(examples_dir, 'OneraM6wing', 'OneraM6_SU2_RANS.plt')
ds = tp.data.load_tecplot(datafile)

## The Dataset
Each dataset in Tecplot 360 is contained within a Frame. The dataset contains all the data available for plotting an analysis.  Access the dataset via a Frame object.

In [3]:
ds = tp.active_frame().dataset # Get a handle to the dataset

In [4]:
print(ds) # Print basic information about the dataset

Dataset: 'Visualization of the volumetric solution | Visualization of the surface solution'
  Zones: 'FluidVolume', 'WingSurface'
  Variables: 'x', 'y', 'z', 'Density', 'Momentum U (Density*U)',
    'Momentum V (Density*V)', 'Momentum W (Density*W)', 'Energy (Density*E)',
    'SA Turbulent Eddy Viscosity', 'Pressure', 'Temperature',
    'Pressure_Coefficient', 'Mach', 'Laminar_Viscosity',
    'Skin_Friction_Coefficient', 'Heat_Flux', 'Y_Plus', 'Eddy_Viscosity'


In [5]:
# Get the number of zones
ds.num_zones

2

In [6]:
# Get the number of variables
ds.num_variables

18

In [7]:
# Get the list of solution times
# This produces an empty list, since this is a steady state solution
ds.solution_times

[]

In [8]:
# Print basic information about each zone
for z in ds.zones():
    print("Name:", z.name)
    print("  Type:", z.zone_type)
    print("  Time:", z.solution_time)
    print("  Num Points:", z.num_points)
    print("  Num Cells:", z.num_elements)

Name: FluidVolume
  Type: ZoneType.FEBrick
  Time: 0.0
  Num Points: 46417
  Num Cells: 43008
Name: WingSurface
  Type: ZoneType.FEQuad
  Time: 0.0
  Num Points: 1453
  Num Cells: 1408


In [9]:
# Print basic information about each variable
for v in ds.variables():
    print("Name:", v.name)
    print("  Index:", v.index)
    print("  Min:", v.min())
    print("  Max:", v.max())

Name: x
  Index: 0
  Min: -6.447196007
  Max: 5.64129591
Name: y
  Index: 1
  Min: 0.0
  Max: 7.643424988
Name: z
  Index: 2
  Min: -6.447196007
  Max: 6.447196007
Name: Density
  Index: 3
  Min: 0.3709256053
  Max: 1.861387014
Name: Momentum U (Density*U)
  Index: 4
  Min: -23.51499939
  Max: 382.894989
Name: Momentum V (Density*V)
  Index: 5
  Min: -106.4394989
  Max: 277.6286926
Name: Momentum W (Density*W)
  Index: 6
  Min: -152.3856049
  Max: 220.5104065
Name: Energy (Density*E)
  Index: 7
  Min: 84192.75
  Max: 392579.4063
Name: SA Turbulent Eddy Viscosity
  Index: 8
  Min: 1.000000013e-10
  Max: 0.03884270042
Name: Pressure
  Index: 9
  Min: 33677.10156
  Max: 146757.0938
Name: Temperature
  Index: 10
  Min: 221.1672974
  Max: 464.5385132
Name: Pressure_Coefficient
  Index: 11
  Min: -1.300665975
  Max: 1.138303995
Name: Mach
  Index: 12
  Min: 0.0
  Max: 1.575580001
Name: Laminar_Viscosity
  Index: 13
  Min: 1.446262013e-05
  Max: 2.538917943e-05
Name: Skin_Friction_Coefficient

### Use indices, names, and wildcards to select a subset of the zones/variables

In [10]:
ds.zone(0) # Zero-based. Gets the 1st zone in the dataset.
ds.zone("FluidVolume") # Get the first zone named "FluidVolume"
ds.zones("*u*") # Get all the zones containing the letter 'u'

ds.variable(3) # Zero-based. Gets the 4th variable in the dataset
ds.variable("Momentum*") # Get the first variable that starts with "Momentum"
ds.variables("Momentum*") # Get all the "Momentum" variables

<generator object Dataset.variables at 0x0000020927308120>

## Accessing Zone Data
The Zone is the basic building block of data in Tecplot 360. Zones can be of many different types (Structured, Unstructured, Polyhedral).  Variable data may also be stored at the nodes or cell centers for each zone/variable pair. That said, accessing the raw variable data is the same for all zone types. 

In [11]:
zone = ds.zone("Fluid*")
var = ds.variable("Temperature")

values = zone.values(var)
# values is a Tecplot array object. From this you can get various information
# about this Zone/Variable pair
print(values)
print(values.data_type)
print(values.location)
print(values.shape)
print(values.minmax())

FieldDataType.Float
ValueLocation.Nodal
(46417,)
(221.1672974, 464.5385132)


In [12]:
# It's best to get values using [:] or 'as_numpy_array'
fast_vals = values[:]
fast_vals = values.as_numpy_array() # This does the same as the line above
print(fast_vals)

[328.666  327.6085 327.1082 ... 288.1656 288.1629 288.1625]


## Modifying Data
PyTecplot can be used to access and modify the raw data in the dataset.  We can also compute new values and add them to the dataset.

In [13]:
# Prefer using Tecplot equations - this executes the equation within 360 and is multi-threaded
tp.data.operate.execute_equation("{Temperature} = {Temperature} * 10")

In [14]:
for z in ds.zones():
    ft_array = z.values("Temperature")
    vals = ft_array[:]
    vals /= 10
    ft_array[:] = vals

In [15]:
import numpy as np
for zone in ds.zones():
    values = zone.values("Temperature")[:]
    avg = np.average(values)
    zone.aux_data["AvgTemp"] = avg

In [16]:
tp.active_frame().add_text("Average Fluid Temp: &(AUXZONE[ActiveOffset=1]:AvgTemp)", position=(5,95))
tp.active_frame().add_text("Average Wing Temp: &(AUXZONE[ActiveOffset=2]:AvgTemp)", position=(5,92))